## Prepare POSCAR files and VASP input files from PDB CO2 configurations

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

from ase import Atoms
from ase.io import read, write
from pymatgen.io.cif import CifParser
from shutil import copy2

plt.rcParams['font.size'] = '15'

In [2]:
# define some parameters
# Constant
# k2kjmol = 0.00831446 # constants. Convert K to kJ/mol

# number of movie frames
nframe = 1

# MOF CONTCAR information
# Take CONTCAR from single point calculations of MOF 
mof_file = "sp_mof211/CONTCAR"

# Movie file
movie_file = "co2_movie_2000_211.pdb"

In [3]:
# read CONTCAR file
mof = read(mof_file)

In [4]:
co2 = [0 for i in range(nframe)]

# Read the movie file into a numpy array
# read PDF file
f = open(movie_file)

# loop over each frame
for i in range(nframe):

    # read first two lines
    line = [next(f).strip().split() for x in range(2)]

    symbols = []
    pos = []
    cell = mof.cell

    while 1:
        line = next(f).strip().split()

        if line[0] == 'ENDMDL':
            break

        symbols.append(line[2])
        pos.append([float(line[4]), 
                    float(line[5]), 
                    float(line[6])])
    
    # create Atoms object
    co2[i] = Atoms(symbols=symbols, positions=pos, cell=cell, pbc=True)


In [5]:
len(co2)

1

In [6]:
# change directory
os.chdir('sp_system')
path0 = os.getcwd()

os.chdir('all_211')
path1 = os.getcwd()

# create a directory for each element in co2 list
for i in range(len(co2)):

    dir_name = str(i)  

    os.chdir(path1)
    os.mkdir(dir_name)

    ##### create a CO2 DFT runs for reference
    os.chdir(dir_name)
    os.mkdir('co2')
    os.chdir('co2')

    # write POSCAR file
    write('POSCAR',co2[i])  
    # copy other input files to this directory
    copy2(path0+'/INCAR', './')
    copy2(path0+'/KPOINTS', './')
    copy2(path0+'/perl_gpu.sh', './')
    copy2(path0+'/POTCAR_co2', './POTCAR')
    copy2(path0+'/vdw_kernel.bindat', './')

    # submit jobs 
    subprocess.run(['sbatch', 'perl_gpu.sh'])

    ##### create a CO2+MOF DFT runs 
    os.chdir(path1+'/'+dir_name)
    os.mkdir('system')
    os.chdir('system')

    # write POSCAR file
    system = mof + co2[i]
    write('POSCAR',system)
    # copy other input files to this directory
    copy2(path0+'/INCAR', './')
    copy2(path0+'/KPOINTS', './')
    copy2(path0+'/perl_gpu.sh', './')
    copy2(path0+'/POTCAR_system', './POTCAR')
    copy2(path0+'/vdw_kernel.bindat', './')
    
    # submit jobs 
    subprocess.run(['sbatch', 'perl_gpu.sh'])

    



Submitted batch job 10473679
Submitted batch job 10473680
